In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 20 14:23:40 2017

@author: JTay
"""


import sklearn.model_selection as ms
from sklearn.ensemble import AdaBoostClassifier
from helpers import dtclf_pruned
import pandas as pd
from helpers import  basicResults,makeTimingCurve,iterationLC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np


cancer = pd.read_csv('./data/breast_cancer.csv')  
cancer = pd.get_dummies(cancer, columns = ['class'], prefix = 'class')
cancer['class'] = cancer['class_2.0']
cancer.drop(['class_2.0', 'class_4.0'], axis = 1, inplace = True)
cancer = cancer.astype(np.int64)

cancerX = cancer.drop('class',1).copy().values
cancerY = cancer['class'].copy().values


contra = pd.read_csv('./data/cmc.data.txt')      
col_names_contra = ['wifes_age', 'wifes_edu', 'husbs_edu', 'num_children_born', 'wifes_religion', 'wife_employed', 'husbs_occup', 'SOL_index', 'media_expose', 'contra_method']
contra.columns = col_names_contra
contra[col_names_contra] = contra[col_names_contra].astype(np.int64)
contra = pd.get_dummies(contra, columns = ['husbs_occup'], prefix = 'husbs_occup')

contraX = contra.drop('contra_method',1).copy().values
contraY = contra['contra_method'].copy().values


alphas = [-1,-1e-3,-(1e-3)*10**-0.5, -1e-2, -(1e-2)*10**-0.5,-1e-1,-(1e-1)*10**-0.5, 0, (1e-1)*10**-0.5,1e-1,(1e-2)*10**-0.5,1e-2,(1e-3)*10**-0.5,1e-3]


contra_trgX, contra_tstX, contra_trgY, contra_tstY = ms.train_test_split(contraX, contraY, test_size=0.3, random_state=0,stratify=contraY)     
cancer_trgX, cancer_tstX, cancer_trgY, cancer_tstY = ms.train_test_split(cancerX, cancerY, test_size=0.3, random_state=0,stratify=cancerY)     



cancer_base = dtclf_pruned(criterion='gini',class_weight='balanced',random_state=55)                
contra_base = dtclf_pruned(criterion='entropy',class_weight='balanced',random_state=55)
OF_base = dtclf_pruned(criterion='gini',class_weight='balanced',random_state=55)                

paramsD = {'Boost__n_estimators':[1,2,5,10,20,30,45,60,80,100],
           'Boost__base_estimator__alpha':alphas}
paramsC = {'Boost__n_estimators':[1,2,5,10,20,30,45,60,80,100],
           'Boost__base_estimator__alpha':alphas}
                                   
         
cancer_booster = AdaBoostClassifier(algorithm='SAMME',learning_rate=1,base_estimator=cancer_base,random_state=55)
contra_booster = AdaBoostClassifier(algorithm='SAMME',learning_rate=1,base_estimator=contra_base,random_state=55)
OF_booster = AdaBoostClassifier(algorithm='SAMME',learning_rate=1,base_estimator=OF_base,random_state=55)


pipeO = Pipeline([('Scale',StandardScaler()),                
                 ('Boost',contra_booster)])
pipeC = Pipeline([('Scale',StandardScaler()),                
                 ('Boost',cancer_booster)])


cancer_clf = basicResults(pipeC,cancer_trgX,cancer_trgY,cancer_tstX,cancer_tstY,paramsC,'Boost','cancer')        
contra_clf = basicResults(pipeO,contra_trgX,contra_trgY,contra_tstX,contra_tstY,paramsD,'Boost','contra')        


cancer_final_params = cancer_clf.best_params_
contra_final_params = contra_clf.best_params_
OF_params = {'Boost__base_estimator__alpha':-1, 'Boost__n_estimators':50}


pipeC.set_params(**cancer_final_params)
pipeO.set_params(**contra_final_params)
makeTimingCurve(cancerX,cancerY,pipeC,'Boost','cancer')
makeTimingCurve(contraX,contraY,pipeO,'Boost','contra')


pipeC.set_params(**cancer_final_params)
iterationLC(pipeC,cancer_trgX,cancer_trgY,cancer_tstX,cancer_tstY,{'Boost__n_estimators':[1,2,5,10,20,30,40,50,60,70,80,90,100]},'Boost','cancer')        
pipeO.set_params(**contra_final_params)
iterationLC(pipeO,contra_trgX,contra_trgY,contra_tstX,contra_tstY,{'Boost__n_estimators':[1,2,5,10,20,30,40,50]},'Boost','contra')                

pipeC.set_params(**OF_params)
iterationLC(pipeC,cancer_trgX,cancer_trgY,cancer_tstX,cancer_tstY,{'Boost__n_estimators':[1,2,5,10,20,30,40,50,60,70,80,90,100]},'Boost_OF','cancer')                
pipeO.set_params(**OF_params)
iterationLC(pipeO,cancer_trgX,cancer_trgY,cancer_tstX,cancer_tstY,{'Boost__n_estimators':[1,2,5,10,20,30,40,50]},'Boost_OF','contra')                

Fitting 5 folds for each of 140 candidates, totalling 700 fits
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=1 ..........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=1, score=0.8282899366643214, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=1 ..........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=1, score=0.9553131597466572, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=1 ..........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=1, score=0.9208304011259677, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=1 ..........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=1, score=0.9066091954022988, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=1 ..........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=1, score=0.9434523809523809, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=2 ..........
[

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
[Parallel(n_jobs

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConve

[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=30, score=0.9208304011259677, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=30 .........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=30, score=0.9066091954022988, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=30 .........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=30, score=0.9434523809523809, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=45 .........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=45, score=0.8282899366643214, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=45 .........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=45, score=0.9553131597466572, total=   0.0s
[CV] Boost__base_estimator__alpha=-1, Boost__n_estimators=45 .........
[CV]  Boost__base_estimator__alpha=-1, Boost__n_estimators=45, score=0.9208304011259677, total=   0.0s
[CV] Boost__base_estimator_

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConve

...


50
60
70
80
90
100
Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] Boost__n_estimators=1 ...........................................
[CV] .. Boost__n_estimators=1, score=0.8659394792399722, total=   0.0s
[CV] Boost__n_estimators=1 ...........................................
[CV] ... Boost__n_estimators=1, score=0.962350457424349, total=   0.0s
[CV] Boost__n_estimators=1 ...........................................
[CV] .. Boost__n_estimators=1, score=0.9208304011259677, total=   0.0s
[CV] Boost__n_estimators=1 ...........................................
[CV] .. Boost__n_estimators=1, score=0.9066091954022988, total=   0.0s
[CV] Boost__n_estimators=1 ...........................................
[CV] .. Boost__n_estimators=1, score=0.9330357142857142, total=   0.0s
[CV] Boost__n_estimators=2 ...........................................
[CV] .. Boost__n_estimators=2, score=0.8659394792399722, total=   0.0s
[CV] Boost__n_estimators=2 .........................................

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConve

[CV] . Boost__n_estimators=10, score=0.8659394792399722, total=   0.0s
[CV] Boost__n_estimators=10 ..........................................
[CV] .. Boost__n_estimators=10, score=0.962350457424349, total=   0.0s
[CV] Boost__n_estimators=10 ..........................................
[CV] . Boost__n_estimators=10, score=0.9208304011259677, total=   0.0s
[CV] Boost__n_estimators=10 ..........................................
[CV] . Boost__n_estimators=10, score=0.9066091954022988, total=   0.0s
[CV] Boost__n_estimators=10 ..........................................
[CV] . Boost__n_estimators=10, score=0.9330357142857142, total=   0.0s
[CV] Boost__n_estimators=20 ..........................................
[CV] . Boost__n_estimators=20, score=0.8659394792399722, total=   0.0s
[CV] Boost__n_estimators=20 ..........................................
[CV] .. Boost__n_estimators=20, score=0.962350457424349, total=   0.0s
[CV] Boost__n_estimators=20 ..........................................
[CV] .

/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/kmertan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConve

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('Scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('Boost', AdaBoostClassifier(algorithm='SAMME',
          base_estimator=dtclf_pruned(alpha=-1, class_weight='balanced', criterion='entropy',
       max_depth=None, max_features=None, max_leaf_nodes=None,
       min_impurity...   random_state=55, splitter='best'),
          learning_rate=1, n_estimators=50, random_state=55))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'Boost__n_estimators': [1, 2, 5, 10, 20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(balanced_accuracy), verbose=10)